In [13]:
import os
import sys
os.getcwd()
import pandas as pd
import numpy as np
import re
import gc
gc.collect()
from pathlib import Path
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

In [6]:
import os
for dirname, _, filenames in os.walk('C:/Users/admin/Documents/Nitin/mycodes/kaggle_google_quest_qna/data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

C:/Users/admin/Documents/Nitin/mycodes/kaggle_google_quest_qna/data\sample_submission.csv
C:/Users/admin/Documents/Nitin/mycodes/kaggle_google_quest_qna/data\test.csv
C:/Users/admin/Documents/Nitin/mycodes/kaggle_google_quest_qna/data\train.csv


In [7]:
%%time
datadir = Path(dirname)

# Read in the data CSV files
train = pd.read_csv(datadir/'train.csv')
test = pd.read_csv(datadir/'test.csv')
sample_submission = pd.read_csv(datadir/'sample_submission.csv')

Wall time: 196 ms


In [ ]:
# Main contents: [question_title, question_body, answer]
#  Extra metadata: [host, url, category, question_user_page, question_user_name, answer_user_page, answer_user_name]

In [8]:
sample_submission.columns
print('total targets', len([a for a in sample_submission.columns if 'question_' in a or 'answer_' in a]))
print('question targets', len([a for a in sample_submission.columns if 'question_' in a]))
print('Answer targets', len([a for a in sample_submission.columns if 'answer_' in a]))

total targets 30
question targets 21
Answer targets 9


In [9]:
train0 = train.iloc[0]

print('URL           : ', train0['url'])
print('question_title: ', train0['question_title'])
print('question_body : ', train0['question_body'])

URL           :  http://photo.stackexchange.com/questions/9169/what-am-i-losing-when-using-extension-tubes-instead-of-a-macro-lens
question_title:  What am I losing when using extension tubes instead of a macro lens?
question_body :  After playing around with macro photography on-the-cheap (read: reversed lens, rev. lens mounted on a straight lens, passive extension tubes), I would like to get further with this. The problems with the techniques I used is that focus is manual and aperture control is problematic at best. This limited my setup to still subjects (read: dead insects) Now, as spring is approaching, I want to be able to shoot live insects. I believe that for this, autofocus and settable aperture will be of great help.

So, one obvious but expensive option is a macro lens (say, EF 100mm Macro) However, I am not really interested in yet another prime lens. An alternative is the electrical extension tubes.

Except for maximum focusing distance, what am I losing when using tubes 

In [11]:
train['question_title_num_chars'] = train['question_title'].apply(lambda x: len(str(x)))
train['question_title_num_words'] = train['question_title'].apply(lambda x: len(str(x).split()))
train['question_body_num_chars'] = train['question_body'].apply(lambda x: len(str(x)))
train['question_body_num_words'] = train['question_body'].apply(lambda x: len(str(x).split()))
train['answer_num_chars'] = train['answer'].apply(lambda x: len(str(x)))
train['answer_num_words'] = train['answer'].apply(lambda x: len(str(x).split()))

test['question_title_num_chars'] = test['question_title'].apply(lambda x: len(str(x)))
test['question_title_num_words'] = test['question_title'].apply(lambda x: len(str(x).split()))
test['question_body_num_chars'] =  test['question_body'].apply(lambda x: len(str(x)))
test['question_body_num_words'] =  test['question_body'].apply(lambda x: len(str(x).split()))
test['answer_num_chars'] =         test['answer'].apply(lambda x: len(str(x)))
test['answer_num_words'] =         test['answer'].apply(lambda x: len(str(x).split()))


In [17]:
tfidf = TfidfVectorizer(ngram_range=(1, 3))
tsvd = TruncatedSVD(n_components=128, n_iter=5)

tfquestion_title = tfidf.fit_transform(train['question_title'].values)
tfquestion_title_test = tfidf.transform(test['question_title'].values)
svdquestion_title = tsvd.fit_transform(tfquestion_title)
svdquestion_title_test = tsvd.transform(tfquestion_title_test)

print(tfquestion_title.shape)
print(tfquestion_title_test.shape)
print(svdquestion_title.shape)
print(svdquestion_title_test.shape)


(6079, 51565)
(476, 51565)
(6079, 128)
(476, 128)
